## Importamos Bibliotecas

In [70]:
# Biblioteca pandas para el manejo de datos
import pandas as pd

# EXTRACCIÓN DE LOS DATOS


In [71]:
# Extraemos la información de cada csv y se convierte en un dataframe
canales = pd.read_csv('..\\data\\canales.csv', encoding='utf-8')
clientes = pd.read_csv('..\\data\\clientes.csv', encoding='utf-8')
transacciones = pd.read_csv('..\\data\\transacciones.csv', encoding='utf-8')

# TRANSFORMACIÓN DE LOS DATOS 

In [72]:
# Eliminamos valores duplicados para canales teniendo en cuenta la columna 'codigo'
canales.drop_duplicates(subset=['codigo'],inplace=True)
# Eliminamos valores duplicados para clientes teniendo en cuenta la columna 'num_doc'
clientes.drop_duplicates(subset=['num_doc'],inplace=True)

In [73]:
canales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32974 entries, 0 to 32973
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   codigo            32974 non-null  int64 
 1   nombre            32974 non-null  object
 2   tipo              32974 non-null  object
 3   cod_jurisdiccion  32974 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.0+ MB


In [74]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 698022 entries, 0 to 701051
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tipo_doc            698022 non-null  object 
 1   num_doc             698022 non-null  int64  
 2   nombre              698022 non-null  object 
 3   tipo_persona        698022 non-null  object 
 4   ingresos_mensuales  698022 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 32.0+ MB


In [75]:
transacciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360614 entries, 0 to 2360613
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   fecha_transaccion  object 
 1   cod_canal          float64
 2   tipo_doc           object 
 3   num_doc            int64  
 4   naturaleza         object 
 5   monto              float64
dtypes: float64(2), int64(1), object(3)
memory usage: 108.1+ MB


In [76]:
# Convertimos la columna de 'fecha_transaccion' de Object a datetime
transacciones['fecha_transaccion'] = pd.to_datetime(transacciones['fecha_transaccion'])

# Convertimos los columnas de datos correspondientes de Object a string
transacciones['tipo_doc'] = transacciones['tipo_doc'].astype("string")
transacciones['naturaleza'] = transacciones['naturaleza'].astype("string")
clientes['tipo_doc'] = clientes['tipo_doc'].astype("string")
clientes['nombre'] = clientes['nombre'].astype("string")
clientes['tipo_persona'] = clientes['tipo_persona'].astype("string")
canales['nombre'] = canales['nombre'].astype("string")
canales['tipo'] = canales['tipo'].astype("string")

# Convertimos la columna de double a int para eliminar los decimales
transacciones['cod_canal'] = transacciones['cod_canal'].astype(int)

# Convertimos las columnas que se pueden convertir numero a string
transacciones['num_doc'] = transacciones['num_doc'].astype("string")
transacciones['cod_canal'] = transacciones['cod_canal'].astype("string")
clientes['num_doc'] = clientes['num_doc'].astype("string")
canales['cod_jurisdiccion'] = canales['cod_jurisdiccion'].astype("string")
canales['codigo'] = canales['codigo'].astype("string")

In [77]:
# Eliminar las filas donde cod_canal no coincide con los valores en df_canales
transacciones = transacciones[transacciones['cod_canal'].isin(canales['codigo'])]

# CARGA DE LOS DATOS

In [78]:
import pyodbc
from sqlalchemy import create_engine

server = 'LAPTOP-E360B96B\SQLEXPRESS'
database = 'Prueba_bancolombia'

engine = create_engine(f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server")

In [79]:
# Insertar datos en la tabla clientes
clientes.to_sql('clientes', engine, index=False, if_exists='append')

# Insertar datos en la tabla canales
canales.to_sql('canales', engine, index=False, if_exists='append')

# Insertar datos en la tabla transacciones
transacciones.to_sql('transacciones', engine, index=False, if_exists='append')


307

# ANÁLISIS DESCRIPTIVO DE LOS DATOS 

In [80]:
# Resumen estadístico general de tabla clientes 
print("Resumen estadístico general de clientes:")
print(clientes.describe())

Resumen estadístico general de clientes:
       ingresos_mensuales
count        6.980220e+05
mean         5.037177e+06
std          2.883255e+06
min          1.850000e+03
25%          2.575704e+06
50%          5.032722e+06
75%          7.528875e+06
max          9.995531e+06


In [81]:
# Resumen estadístico general de la tabla transacciones
print("Resumen estadístico general de transacciones:")
print(transacciones.describe())

Resumen estadístico general de transacciones:
                   fecha_transaccion         monto
count                        2360594  2.360594e+06
mean   2024-05-28 22:52:07.997105664  5.055470e+06
min              2024-01-01 00:00:00  1.000000e+00
25%              2024-03-04 00:00:00  1.000000e+05
50%              2024-06-16 00:00:00  3.000000e+05
75%              2024-08-09 00:00:00  1.000000e+06
max              2024-10-07 00:00:00  4.775500e+10
std                              NaN  1.633161e+08


In [82]:
# Número total de transacciones por cliente
transacciones_por_cliente = transacciones.groupby('num_doc').size()
print("\nNúmero total de transacciones por cliente:")
print(transacciones_por_cliente)


Número total de transacciones por cliente:
num_doc
-1000076379879933291     9
-1000147930647869782    55
-1000460014616510321    13
-1000951368134169935     9
-100101085567707252     12
                        ..
999050126532234206      13
999075441145575820      78
999126550605799708      33
999389099053346505      54
999575352248400936      19
Length: 70553, dtype: int64


In [83]:
# Monto total de transacciones por cliente
monto_total_por_cliente = transacciones.groupby('num_doc')['monto'].sum()
print("\nMonto total de transacciones por cliente:")
print(monto_total_por_cliente)


Monto total de transacciones por cliente:
num_doc
-1000076379879933291      2230000.0
-1000147930647869782     21215000.0
-1000460014616510321     10390000.0
-1000951368134169935      5070000.0
-100101085567707252       6120000.0
                           ...     
999050126532234206      345542384.0
999075441145575820        6270000.0
999126550605799708       60770000.0
999389099053346505       10860000.0
999575352248400936        2775000.0
Name: monto, Length: 70553, dtype: float64


In [84]:
# Distribución de los montos de las transacciones
print("\nDistribución de los montos de las transacciones:")
print(transacciones['monto'].describe())


Distribución de los montos de las transacciones:
count    2.360594e+06
mean     5.055470e+06
std      1.633161e+08
min      1.000000e+00
25%      1.000000e+05
50%      3.000000e+05
75%      1.000000e+06
max      4.775500e+10
Name: monto, dtype: float64


In [85]:
# Media y mediana de ingresos mensuales por tipo de persona
ingresos_por_tipo_persona = clientes.groupby('tipo_persona')['ingresos_mensuales'].agg(['mean', 'median'])
print("\nMedia y mediana de ingresos mensuales por tipo de persona:")
print(ingresos_por_tipo_persona)


Media y mediana de ingresos mensuales por tipo de persona:
                          mean     median
tipo_persona                             
-                 5.180662e+06  5177504.0
NATURAL           4.814551e+06  5079649.5
PERSONA JURIDICA  5.072947e+06  5082839.0
PERSONA NATURAL   5.035913e+06  5028636.0


In [86]:
# Media y mediana del monto total de transacciones por tipo de persona
monto_total_por_tipo_persona = transacciones.groupby('tipo_doc')['monto'].agg(['mean', 'median'])
print("\nMedia y mediana del monto total de transacciones por tipo de persona:")
print(monto_total_por_tipo_persona)


Media y mediana del monto total de transacciones por tipo de persona:
                                                   mean     median
tipo_doc                                                          
-                                          1.417000e+08  2511847.5
CEDULA DE CIUDADANIA                       1.053876e+06   300000.0
CEDULA DE EXTRANJERIA                      1.368191e+06   300000.0
DOCUMENTO VENEZOLANO/CARNET DIPLOMÁTICO    1.228372e+06   400000.0
ID EXTRANJERO PN NO RESIDENTE EN COLOMBIA  9.313810e+05   550000.0
NIT                                        1.013541e+07   865535.0
PASAPORTE                                  1.149261e+06   400000.0
REGISTRO CIVIL                             7.113704e+05   250000.0
TARJETA DE IDENTIDAD                       6.556301e+05   200000.0


In [87]:
# Frecuencia de uso de cada canal
frecuencia_canales = transacciones['cod_canal'].value_counts()
print("\nFrecuencia de uso de cada canal:")
print(frecuencia_canales)


Frecuencia de uso de cada canal:
cod_canal
1000598    13254
1000379     2461
2030974     2367
1000171     1547
1000688     1496
           ...  
2072865        1
2075173        1
2068324        1
2059744        1
2059978        1
Name: count, Length: 32974, dtype: Int64


In [88]:
# Distribución de montos de transacciones por canal
monto_por_canal = transacciones.groupby('cod_canal')['monto'].describe()
print("\nDistribución de montos de transacciones por canal:")
print(monto_por_canal)


Distribución de montos de transacciones por canal:
           count          mean           std      min        25%        50%  \
cod_canal                                                                     
1000001    486.0  1.238511e+07  2.907473e+07    100.0   893753.5  3785254.0   
1000002    255.0  1.207875e+07  2.096226e+07   2000.0  1946500.0  4700000.0   
1000004    467.0  1.066075e+07  2.550484e+07  47119.0  1322770.0  4000000.0   
1000005    536.0  1.741924e+07  4.978748e+07  20000.0  1285000.0  3500000.0   
1000006    334.0  1.583475e+07  7.578914e+07  27726.0   957250.0  2956500.0   
...          ...           ...           ...      ...        ...        ...   
3009985     99.0  5.376768e+05  6.126237e+05  10000.0   100000.0   300000.0   
3009986     27.0  5.755556e+05  6.436992e+05  20000.0   100000.0   450000.0   
3009987    212.0  5.725943e+05  4.859069e+05  20000.0   150000.0   500000.0   
3009988     85.0  3.484706e+05  3.975895e+05  20000.0   100000.0   200000.0   
